In [1]:
import os
os.environ['HF_HOME'] = 'e:/.cache/huggingface'

from pathlib import Path
import shutil
from dataclasses import dataclass

from tqdm.auto import tqdm
import gradio as gr
import datasets
from datasets import load_dataset

In [2]:
dataset_dir = Path('datasets/yodas2_ru000_16k')
yodas2 = load_dataset(str(dataset_dir), split='train')

Resolving data files:   0%|          | 0/34042 [00:00<?, ?it/s]

In [3]:
database = {
    i: ' '.join(sample['utterances']['text'])
    for i, sample in enumerate(tqdm(yodas2))
}
len(database)

  0%|          | 0/33542 [00:00<?, ?it/s]

33542

In [4]:
sum(len(x) for x in database.values()) / 10**6

155.20702

In [ ]:
def find(database: dict[int | str, str], query: str | list[str], max_count: int | None = None) -> list[str]:
    if isinstance(query, str):
        query = [query]
    query = [q.strip().lower().replace('ё', 'е') for q in query]
    found_ids = []
    for id, text in database.items():
        if max_count is not None and len(found_ids) >= max_count:
            break
        text = text.lower()
        if all([(q in text if not q.startswith('-') else q not in text) for q in query]):
            found_ids.append(id)
    return found_ids

In [ ]:
query = ['подойдем к']

output_dir = Path('datasets/yodas_found')

max_found = 9999
max_written = 100

found_ids = find(database, query, max_count=max_found)
print(f'Found {len(found_ids)} results.')

shutil.rmtree(output_dir, ignore_errors=True)
output_dir.mkdir(parents=True, exist_ok=True)

for id in found_ids[:max_written]:
    sample = yodas2[id]
    youtube_id = sample['video_id']
    rel_audio_path = Path(sample['audio']['path'])
    shutil.copy(dataset_dir / rel_audio_path, output_dir / rel_audio_path.name)

    text = ' '.join(sample['utterances']['text'])
    sep = '================================'
    print(f'\n{sep}\n{youtube_id}\n{sep}\n{text[:2000]}')

In [ ]:
print(article_ar)

In [ ]:
text = 'Hello everyone, you are on Seoquick channel, my name is Nikolai Shmichkov, and I am Ulitovsky Anatoly, and today we will answer the age-old question how to calculate the cost of promoting your site? I will briefly talk about the schemes, Anatoly will tell us about his methods, and we will approach the general view. I am starting today, and so I want to share a simple system. I remind you that the cost of SEO promotion is primarily composed of three components, the first component, of course, is the salary of seamstresses. You must pay for the work, of course, those who advise you, those who help you, those who give you strategies, give you plans. That is the same work that is in our interest. Anyone who is engaged in SEO, be it your staff member, be it you hired a freelance'
print(text)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

device = 'cuda' #or 'cpu' for translate on cpu

model_name = 'utrobinmv/t5_summary_en_ru_zh_base_2048'
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()
model.to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)

text = """мы заходим в парк Политехнического университета. Водонапорная башня, сейчас проходим мимо неё Санкт-Петербургский Политехнический университет был основан 19 февраля 1899 года в соответствии с поручением министр финансов Российской Империи Витте. Историю и славу Политехнического университета в течение более 100 лет создавали люди, которые в нем преподавали и учились. Университет выпустил ряд выдающихся ученых: академики Абрам Иоффе, Игорь Курчатов, лауреат нобелевской премии Николай Семенов, Жорес Алфёров и Петр Капица. В 1918 году работа института была приостановлена, после Октябрьской революции многие преподаватели покинули Санкт-Петербург и Россию, но несмотря на это в марте 1919 года был создан первый в мировой истории физико-механический факультет для подготовки инженеров физиков-исследователей. В годы Великой Отечественной войны институт понес огромные людские потери, на фронт ушли более половины студентов, преподавателей и сотрудников университета. В 1967 году в парке Политехнического университета был установлен памятник погибшим "политехникам", но самое интересное что лучший в мире массовый средний танк времен Второй Мировой войны, всем нам известный танк Т-34 был создан выпускником Политехнического университета- Михаилом Ильичом Кошкиным. "ребята поменьше сложностей, делайте так, чтобы машина была доступна любому механику"- Михаил Ильич Кошкин, "политехник". Экипаж: 4 человека; боевая масса 30,9 тонн; длина 6,62 метра ; ширина 3 метра; высота 2,52; количество пушек 1; калибра 76 миллиметров; количество пулеметов 2 ; калибр 7,62 ; броня лобовая 45 миллиметров; броня бортовая 45 миллиметров; двигатель дизельный 450 лошадиных сил; максимальная скорость 51 камер час; запас хода 300 километров -Знаете, что зжесь упокоился Григорий Распутин? -Сожгли, да? -Да, вот здесь в этой котельне. -да да я что-то такое слышала. Попытались сначала по пути сжечь не получилось до конца вот сюда привезли останки Распутина похоронили 21 декабря 1916 года на территории царского сел
"""

inputs = tokenizer('summary brief: ' + text, return_tensors="pt")
tokenizer.batch_decode(model.generate(**inputs.to(device)), skip_special_tokens=True)